Übung: Linux Namespaces - nsenter
---------------------------------

![](images/linuxns.png)

- - -

Linux Namespaces sind eine Technologie, die es ermöglicht, verschiedene Systemressourcen wie Prozesse, Netzwerkinterfaces oder Dateisysteme voneinander zu isolieren. 

Dies schafft die Grundlage für Containerisierung, bei der mehrere isolierte Umgebungen auf einem einzigen Host-System laufen können, ohne sich gegenseitig zu beeinflussen. 

**Verschiedene Linux Namespaces**
* Prozess Namespace: Isoliert die Prozess-IDs, sodass Prozesse in verschiedenen PID-Namespaces die gleichen IDs haben können, ohne Konflikte zu verursachen.
* Network Namespace: Trennt die Netzwerkstacks, inklusive Interfaces, Routing-Tabellen und Ports, was unterschiedliche Netzwerkkonfigurationen in verschiedenen Namespaces ermöglicht.
* Mount Namespace: Isoliert die Mountpoints von Dateisystemen, sodass jeder Namespace eine eigene Sicht auf das Dateisystem hat.
* UTS Namespace: Trennt die Kernel- und Hostnamen, sodass jeder Namespace eigene Werte für den Host- und Domainnamen haben kann.
* IPC Namespace: Isoliert die Inter-Prozess-Kommunikationsmechanismen wie Message Queues, Semaphoren und Shared Memory, sodass sie nur innerhalb eines Namespaces sichtbar sind.
* User Namespace: Trennt die User- und Gruppen-IDs, was es ermöglicht, dass Benutzer in verschiedenen Namespaces unterschiedliche Privilegien haben können.
* Control Groups (Cgroups): Eine Linux-Kernel-Funktion, die verwendet wird, um die Ressourcenverwendung von Prozessen zu verwalten und zu limitieren.

- - -

Mit **nsenter** können Administratoren und Entwickler direkt in die isolierten Umgebungen eines Containers wechseln, um Debugging, Wartung oder Konfigurationsänderungen durchzuführen.

- - -

Um die Funktionalität von Linux Namespaces und nsenter zu demonstrieren, starten wir zunächst einen Container:


In [ ]:
! docker run --rm -d --name webshop-nsenter registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0

Diesen Container sehen wir im Prozesstree mit Namen `webshop` als Subprozess von `containerd`

In [ ]:
! pstree -npTA

Mittels der Prozess-Id `$(pgrep -o -x webshop)` wechseln wir mit `nsenter` in den Container und überprüfen ob dieser 
* einen eigenen Netzwerkadapter hat
* nur seinen eigenen Prozess sieht
* und sein Filesystem eine Alpine Linux Distribution beinhaltet

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --net ip addr

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --all pstree -p

In [ ]:
! sudo nsenter -t $(pgrep -o -x webshop) --all cat /etc/issue

Welche Linux Namespaces unser Container verwendet, können wir mittels `lsns` sichtbar machen:

In [ ]:
! sudo lsns | grep $(pgrep -o -x webshop)

Zusatzübung:
* ersetzt `--all` in den letzten zwei Befehlen durch `--net`
* Was ist anders?

Für weitere Varianten siehe Manuel von `nsenter`

In [ ]:
! man nsenter

- - -
Zur Gegenprobe schauen wir uns Netzwerkadapter und Linux Distribution der **VM** an:

In [ ]:
! ip addr

In [ ]:
! cat /etc/issue

- - -
Die gleiche Funktionalität bietet auch `exec`, jedoch werden immer alle Linux Namespaces gewechselt:

In [ ]:
! docker exec webshop-nsenter ip addr
! echo "----------------------------------"
! docker exec webshop-nsenter pstree -p
! echo "----------------------------------"
! docker exec webshop-nsenter cat /etc/issue

- - -
Aufräumen

In [ ]:
! docker stop webshop-nsenter
! docker image rm registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0

- - - 

### Quellen

* Sourcecode: https://gitlab.com/ch-mc-b/autoshop/shop
* Container Registry: https://gitlab.com/ch-mc-b/autoshop/shop/container_registry
